In [25]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

!pip install tensorflow_text

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [26]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import tensorflow_hub as hub
import numpy as np
import tensorflow_text
import tensorflow as tf
import sklearn.model_selection

Data Preparation

In [ ]:
data = pd.read_csv("/content/drive/Shareddrives/CapBatu/ML Divsion/Sentiment Analysis/data/processed/data.csv")
data

,text,labels
0,friendly staff fantastic location to downtown ...,1
1,lighting in the room was very dark,0
2,the facilities are not that much there is no r...,0
3,cleanliness very kind staff position,1
4,ac during winter,0
...,...,...
742607,except the location we had to spent to get to ...,0
742608,cramped rooms bathrooms and dining room,0
742609,hotel was near to metro,1
742610,meters from tram station a little bit far park...,0


In [ ]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(data["text"], data["labels"], test_size=0.1, random_state=1, stratify=data["labels"])
X_train = X_train.to_list()
y_train = y_train.to_list()

X_test = X_test.to_list()
y_test = y_test.to_list()

 Model Creation

In [ ]:
# embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")

In [ ]:
# english_sentences = ["I enjoy taking long walks along the beach with my dog what the fuck", "ayo what are you doing"]
# en_result = embed(english_sentences)
# en_result.shape

In [ ]:
tf.keras.backend.clear_session()
model = tf.keras.Sequential()
model.add(hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3", input_shape=[], dtype=tf.string, trainable=False))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation="relu"))
model.add(tf.keras.layers.Dense(1, activation="sigmoid"))

In [ ]:
model.compile(optimizer="adam", loss=tf.keras.losses.BinaryCrossentropy(), metrics="accuracy")

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 512)               68927232  
                                                                 
 flatten (Flatten)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 128)               65664     
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 68,993,025
Trainable params: 65,793
Non-trainable params: 68,927,232
_________________________________________________________________


In [ ]:
model.fit(X_train, y_train, batch_size=32, epochs=5, validation_data=(X_test, y_test),
          callbacks=[tf.keras.callbacks.TensorBoard(tensorboard_logdir, histogram_freq=1)])

Epoch 1/5
20886/20886 [==============================] - 695s 33ms/step - loss: 0.1605 - accuracy: 0.9416 - val_loss: 0.1553 - val_accuracy: 0.9433
Epoch 2/5
20886/20886 [==============================] - 694s 33ms/step - loss: 0.1453 - accuracy: 0.9482 - val_loss: 0.1502 - val_accuracy: 0.9463
Epoch 3/5
20886/20886 [==============================] - 691s 33ms/step - loss: 0.1390 - accuracy: 0.9508 - val_loss: 0.1472 - val_accuracy: 0.9477
Epoch 4/5
20886/20886 [==============================] - 662s 32ms/step - loss: 0.1345 - accuracy: 0.9526 - val_loss: 0.1481 - val_accuracy: 0.9468
Epoch 5/5
20886/20886 [==============================] - 657s 31ms/step - loss: 0.1309 - accuracy: 0.9541 - val_loss: 0.1459 - val_accuracy: 0.9481


In [ ]:
tf.saved_model.save(model, "/content/drive/Shareddrives/CapBatu/ML Divsion/Sentiment Analysis/models/v1/saved_model")

INFO:tensorflow:Assets written to: /content/drive/Shareddrives/CapBatu/ML Divsion/Sentiment Analysis/models/saved_model/assets


INFO:tensorflow:Assets written to: /content/drive/Shareddrives/CapBatu/ML Divsion/Sentiment Analysis/models/saved_model/assets


Test Load Model

In [27]:
import tensorflow as tf
import tensorflow_text

In [37]:
saved_model_path = "/content/drive/Shareddrives/CapBatu/ML Divsion/Sentiment Analysis/models/v1/saved_model" # The last one should be "/saved_model"
out_zip_path = "/content/drive/Shareddrives/CapBatu/ML Divsion/Sentiment Analysis/models/v1/v1.zip" # With .zip extension

In [30]:
model = tf.saved_model.load(saved_model_path)

In [36]:
prediction = model(["The bathroom is so smelly", "Room is spacious", "Love the room", "Good scenery", "Toilet is not clean", "Toilet is clean"])
output = tf.reshape(tf.cast(tf.math.round(prediction), dtype="int32"), [len(prediction)]).numpy()
output

array([0, 1, 1, 1, 0, 1], dtype=int32)

In [42]:
!cp -r "{saved_model_path}" "./" && zip -Z bzip2 -r "{out_zip_path}" "./saved_model" && rm -fr "saved_model"

  adding: saved_model/ (stored 0%)
  adding: saved_model/assets/ (stored 0%)
  adding: saved_model/saved_model.pb (bzipped 72%)
  adding: saved_model/variables/ (stored 0%)
  adding: saved_model/variables/variables.index (bzipped 79%)
  adding: saved_model/variables/variables.data-00000-of-00001 (bzipped 5%)
